In [1]:
import os
import ast
import string
from gensim.models import Word2Vec
import re
import gensim
import numpy
import torch

In [2]:
class CornellData:
    def __init__(self):
        dirName='data/'
        self.lines = {}
        self.conversations = []

        MOVIE_LINES_FIELDS = ["lineID","characterID","movieID","character","text"]
        MOVIE_CONVERSATIONS_FIELDS = ["character1ID","character2ID","movieID","utteranceIDs"]

        self.lines = self.loadLines(os.path.join(dirName, "movie_lines.txt"), MOVIE_LINES_FIELDS)
        self.conversations = self.loadConversations(os.path.join(dirName, "movie_conversations.txt"), MOVIE_CONVERSATIONS_FIELDS)

    def loadLines(self, fileName, fields):
        lines = {}
        with open(fileName, 'r', encoding='iso-8859-1') as f:
            for line in f:
                values = line.split(" +++$+++ ")
                values = [''.join(c for c in s if c not in string.punctuation) for s in values]
                # Extract fields
                lineObj = {}
                for i, field in enumerate(fields):
                    lineObj[field] = values[i]
                lines[lineObj['lineID']] = lineObj['text'].strip('\n')
        return lines
     
    def loadConversations(self, fileName, fields):
        conversations = []
        with open(fileName, 'r', encoding='iso-8859-1') as f:  
            for line in f:
                values = line.split(" +++$+++ ")
                # Extract fields
                convObj = {}
                for i, field in enumerate(fields):
                    convObj['utteranceIDs'] = values[i]
                lineIds = ast.literal_eval(convObj["utteranceIDs"])
                convObj["lines"] = []
                for lineId in lineIds:
                    convObj["lines"].append(self.lines[lineId]),
                conversations.append(convObj)
            conversations_list = [d.get('lines') for d in conversations]
            
            lenght=5
            co=[c for c in conversations_list if len(c) > lenght]
            co2 = co[:25]  #temporaneo, sarà poi co direttamente.
            
        return co2
    
    def getConversations(self):
        return self.conversations

In [3]:
dirName='data/'
conversazioni=[]
conversazioni=CornellData()

In [4]:
MOVIE_CONVERSATIONS_FIELDS = ["character1ID","character2ID","movieID","utteranceIDs"]

listaconversazioni = conversazioni.loadConversations(os.path.join(dirName, "movie_conversations.txt"), MOVIE_CONVERSATIONS_FIELDS)

In [5]:
#model = gensim.models.KeyedVectors.load_word2vec_format('/media/daniele/AF56-12AA/GoogleNews-vectors-negative300.bin', binary=True)
model = gensim.models.KeyedVectors.load_word2vec_format('/Users/lucatosto/Desktop/Intelligenza_artificiale/GoogleNews-vectors-negative300.bin', binary=True)

In [6]:
def vettorizzazione():
    token = torch.FloatTensor(1,300)
    token.fill_(0)
    
    uno = torch.FloatTensor(1,1)
    uno.fill_(1)

    zero = torch.FloatTensor(1,1)
    zero.fill_(0)

    inizio = torch.cat([uno, zero], 1)
    fine = torch.cat([zero, uno], 1)

    fineparola = torch.cat([zero, zero], 1)

    iniziobattuta = torch.cat([token, inizio], 1)

    finebattuta = torch.cat([token, fine], 1)
    
    vettorefinale=[]
    for conversazione in listaconversazioni:
        vettorebattute=[]
        for battuta in conversazione:
            vettoreparole=torch.FloatTensor()
            vettoreparole = torch.cat([iniziobattuta, vettoreparole])
            for parola in battuta:
                try:
                    p = model[parola]
                    p = torch.from_numpy(p)
                    p = p.view(1, 300)
                except:
                    pass
                parolatokenizzata = torch.cat([p, fineparola], 1)
                vettoreparole = torch.cat([vettoreparole, parolatokenizzata])
                #vettoreparole = vettoreparole.view(len(battuta),302)
            vettoreparole = torch.cat([vettoreparole, finebattuta])
            vettorebattute.append(vettoreparole)
        vettorefinale.append(vettorebattute)
    
    return vettorefinale       

In [7]:
vett = vettorizzazione()
print(len(vett))

25


In [8]:
vettore_train_sources=vett[:20]
vettore_test_sources=vett[21:25]
print(len(vettore_train_sources))
print(len(vettore_test_sources))

20
4


In [12]:
#save model of train set
torch.save(vettore_train_sources, '/Users/lucatosto/Desktop/Intelligenza_artificiale/ChatBot/train_test/train/sources.txt')
#torch.save(vettore_train_sources, '/home/daniele/Scrivania/train_test/train/sources.txt')
#save model of test set
torch.save(vettore_test_sources,'/Users/lucatosto/Desktop/Intelligenza_artificiale/ChatBot/train_test/test/sources.txt')
#torch.save(vettore_test_sources,'/home/daniele/Scrivania/train_test/test/sources.txt')